plot_change.py: Generate significance of electrode weight changes at neuroanatomical sites

## Pseudocode:
    . Import
    . Input symptom, mode, perdur
    . Define functions
    . Load Parameters
    . For Loop through seizures

## Import

In [1]:
import csv
import os
import pathlib
import numpy as np
from scipy.io import loadmat
from scipy.signal import butter,filtfilt
from scipy.stats import ttest_1samp

#
import pandas as pd
# from plot_change_module import *

In [2]:
neuroanat_list = ['frontalpole', #FRONTAL LOBE
    'parstriangularis',
    'parsopercularis',
    'parsorbitalis',
    'rostralmiddlefrontal',
    'caudalmiddlefrontal',
    'lateralorbitofrontal',
    'superiorfrontal',
    'medialorbitofrontal',
    'precentral',
    'postcentral', # PARIETAL LOBE
    'inferiorparietal',   
    'superiorparietal',
    'supramarginal',
    'temporalpole', # TEMPORAL LOBE
    'middletemporal',
    'superiortemporal',
    'inferiortemporal',
    'parahippocampal',               
    'Right-Hippocampus',
    'Left-Hippocampus',
    'Right-Amygdala',
    'Left-Amygdala',
    'entorhinal',
    'bankssts',
    'fusiform', # OCCIPITAL LOBE                
    'lingual'];
    # 'Right-Inf-Lat-Vent', # OTHER
    # 'Right-Cerebral-White-Matter',
    # 'Left-Cerebral-White-Matter',
    # 'Right-choroid-plexus',
    # 'Right-Putamen',
    # 'Right-VentralDC'];

## Input symptom, mode, and duration

In [3]:
sx_input = 'lhx'
mx_input = '2'
perdur_input = '10'

## Define Functions

In [4]:
def sem_w8s(LL_s,at_onset,before_onset,after_onset):
    print('mr.longtime sem_w')

    row_num = np.shape(LL_s)[0]
    LL_meandiff = np.empty(row_num,)
    for row in range(0,row_num-1): #for each channel in LL   
        LL_meandiff[row] = np.mean(LL_s[row,int(at_onset):int(after_onset)]) - np.mean(LL_s[row,int(before_onset):int(at_onset)])

    return LL_meandiff

In [5]:
# def ll_sort(LL,anat,neuroanat_list):
#     print('mr.longtime ll_s')

    
# #     u_anat = np.unique(anat)
# #     u_anat_i = len(u_anat)
# #     print(u_anat_i)

    
    
    
#     a_count = 0
#     n_count = 0
#     y_count = 0
    
#     n_list = []
#     y_list = []
#     LL_s = np.empty([np.shape(anat)[0],np.shape(LL)[1]])
#     LL_s[:] = np.NaN
    
#     for n_a in neuroanat_list:
#         a_count = 0
#         for a_c in anat:
#             if np.size(a_c) > 0: 
#                 if n_a == a_c:
#                  #collect ll data
#                     n_list.append(n_a) #collect all neuroanat str
                    
#                     LL_s[n_count,:] = LL[a_count,:]
                    
#                     # print(a_c)
# #                     print(LL[a_count,0])
                    
                    
# #                     print(n_list[n_count])
# #                     print(LL_s[n_count,0])
#                     n_count += 1
            
#                     #sort into unique y tick labels
#                     if n_a not in y_list:
#                         y_list.append(n_a) #collect only non-repeating neuroanat str  
#                         # y_count += 1
#                         #find index of first and last appearance of neuroanat str, divide by two for ytick
#             a_count += 1
#     # print(a_count)
#     return LL_s,n_list,y_list                     

In [6]:
def ll_transform(llw,fs,d,bl_start,bl_stop):
    print('mr.longtime ll_t')
    
    if bl_start == 0:
        sample_bl_start = 0
    elif bl_start > 0:
        sample_bl_start = (fs*bl_start)[0] #0 because it's python not matlab
    sample_bl_end = (fs*bl_stop)[0]-1
    
    L = int(np.round(llw * fs) - 1) # number of samples to calculate line length
    
    col_len = np.shape(d)[1]-L
    
    LL = np.empty([np.shape(d)[0],col_len])
    
    LL[:] = np.NaN        
    
    for col_1 in range(0,col_len):
        LL[:,col_1] = np.sum(np.abs(np.diff(d[:,col_1:col_1+L])),1)


    for row_1 in range(0,np.shape(d)[0]):    
        LL_nanmean = np.nanmean(LL[row_1,sample_bl_start:sample_bl_end])
        LL_nanstd = np.nanstd(LL[row_1,sample_bl_start:sample_bl_end])
                                              
        LL[row_1,:] = (LL[row_1,:] - LL_nanmean)/LL_nanstd
        
            
    return LL


In [7]:
def open_xl(xl_name): 
    
    df = pd.read_excel(xl_name, index_col=0, engine='openpyxl')

    return df


In [8]:
def filt(d,fs):
    d_t = d.transpose()
    
    butter_array = np.array([1,(round(fs[0]/2)-1)])
    b,a = butter(2,butter_array/(fs[0]/2),btype='bandpass',output='ba')
    
    filt_d = filtfilt(b,a,d_t,axis=0,padtype='odd',padlen=3*(max(len(b),len(a))-1)).transpose()

    return filt_d


In [9]:
def load_elecs_anat(pt_path):
                
    #load anatomy and electrode matrix
    os.chdir(pt_path + 'Imaging/elecs')
    e_mat = loadmat('clinical_elecs_all.mat')
    anat_col = e_mat['anatomy'][:,3]
            
    return anat_col


In [10]:
def get_params(df_params,pt):
    params_bl_start = df_params.loc[pt]['BLstart']
    params_bl_stop = df_params.loc[pt]['BLstop']
    params_llw = df_params.loc[pt]['llw']

    return params_bl_start, params_bl_stop, params_llw

In [11]:
def get_ptsz(avg_change_path): #Load list of patient and seizure names
    os.chdir(avg_change_path)
    
    df_params = open_xl('OPSCEAparams.xlsx') # Ndimensions and params_list

    with open('sz_name_list.csv','r') as sz_names:
        sz_names_list = list(csv.reader(sz_names))
        sz_rows = np.size(sz_names_list,0) #find row num 
        pt_data = []
        sz_data= []
        for r in range(0,sz_rows):  #collect symptom data for specfic seizure in list
            pt_data.append(sz_names_list[r][0]) 
            sz_data.append(sz_names_list[r][1]) 
    return pt_data, sz_data, df_params

## For loop through patients in list matching seizure

In [12]:
avg_change_path = '/Users/nataliasucher/Desktop/UCSF/Coding/OPSCEA/OPSCEADATA/avg_change_folders/'

pt_data,sz_data,df_params = get_ptsz(avg_change_path) #Ehsan

# for neuro_struc in neuroanat_list:

#For loop through patients
for i in range(len(pt_data)):  #Ehsan
    pt_i = pt_data[i]          #Ehsan
    sz_name = sz_data[i]       #Ehsan

    pt_path = avg_change_path + pt_i + '/'
    os.chdir(pt_path)
    pt_dir = os.listdir(pt_path)

    for sz_name in sz_data: 

        if sz_name in pt_dir: #patients in list matching seizure
 
            print(sz_name)
            
            # PARAMS
            blstart, blstop, llw = get_params(df_params,pt_i) # 2 = params_llw

            # ANAT 
            anat = load_elecs_anat(pt_path)
            
            
            # SEMIOLOGY
            sz_path = pt_path + sz_data[i]
            os.chdir(sz_path)

            sx_vec = pd.read_csv(sz_name + '_mat.csv',usecols = [sx_input]).values.tolist() #load semiology matrix .csv             #WANT TO JUST LOAD 1 COLUMN SX_INPUT            
            
            if np.float64(mx_input) in sx_vec:
            
                sz_mat = loadmat(sz_name +'.mat') #load frame speed and ppEEG
                fs = sz_mat['fs'].flatten()
                # d = sz_mat['ppEEG'][0:np.shape(anat)[0],(fs*blstart)[0]:] 
                d = sz_mat['ppEEG'][0:np.shape(anat)[0],:]
                
                badch_mat = loadmat(sz_name + '_badch.mat')
                badch = badch_mat['bad_chs']
                
                badch = np.delete(badch,range(np.shape(anat)[0],np.shape(badch)[0]),0)

                for a_i in range(0,np.shape(anat)[0]):
                    if np.size(anat[a_i]) > 0: 
                        if anat[a_i][0] not in neuroanat_list:
                            badch[a_i][0] = 1 #prepare for anatomy not in neuroanat list to be deleted in d and anat
                
                bad_logical = np.any(badch,1)
                d = d[~bad_logical,:]
                anat = anat[~bad_logical]

                filt_d = filt(d,fs) #Filter out < 1 Hz (and up to nyquist)
    
                LL = ll_transform(llw,fs,filt_d,blstart,blstop) 
            
                #                 
                at_onset = np.round(((sx_vec.index(np.float64(mx_input)))/5) * fs)
                before_onset = np.round(((sx_vec.index(np.float64(mx_input))/5) - float(perdur_input)) * fs)
                after_onset = np.round(((sx_vec.index(np.float64(mx_input))/5) + float(perdur_input)) * fs)

                LL_meandiff = sem_w8s(LL,at_onset,before_onset,after_onset)
                
                print(np.shape(LL_meandiff))
                
                pval = np.shape(len(neuroanat_list))
                pval[:] = np.NaN

                anat_w8s = np.empty(len(neuroanat_list))
                anat_w8s = np.NaN
                
                for n_l in range(0,len(neuroanat_list)):
                    if neuroanat_list[n_l] is in anat:
                        #clump together all the ones that match right amygdala
                    
                        #STRING COMPARISON 
                        anat_index = strcmpi neuroanat_list[n_l] & anat
                        anat_w8s = LL_meandiff[anat_index]
                        tstat, pval[n_l] = ttest_1samp(anat_w8s,0)
                        plot() #red if significant 
        #subplot outside the for loop 
        #nl = x val, anat_w8s
         

EC96_01
mr.longtime ll_t
mr.longtime sem_w
(96,)


TypeError: 'tuple' object does not support item assignment